In [ ]:
import os
from keras.models import *
from keras.optimizers import *
from keras.layers import *
import numpy as np
import sys
from DataProcessor import *

#the main function of the Eval Model

def ApplyRNNModelEvaluation(X,Y,FGC_Dic,collectors):

    #hidden layers and batch sizes
    HIDDEN_SIZE1 = 512
    HIDDEN_SIZE2 = 256
    BATCH_SIZE = 64


    print('Build model...')

    #use LSTM layers
    RNN = LSTM
    model = Sequential()

    #First RNN layer with input (number of collectors * len of the sequence)
    model.add(RNN(HIDDEN_SIZE1, input_shape=(len(X[0]), len(X[0][0]))))
    model.add(Reshape(target_shape=(16,32)))
    #Second RNN layer with the same input again
    model.add(RNN(HIDDEN_SIZE2, input_shape=(len(X[0]), len(X[0][0]))))
    model.add(Dropout(0.4))
    #last Fully connected layer
    model.add(Dense(HIDDEN_SIZE2))
    model.add((Dense(len(Y[0]))))
    #Softmax to select based on probabilities
    model.add(Activation('softmax'))

    #optimize using gradient descent
    sgd = SGD(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    #to view a summary that represents the full RNN model
    model.summary()

    #load weights of the saved model
    model.load_weights('model')

    #get prediction proabilities for each collector
    predictions=model.predict(X)

    #generate the results file (probabilities for all FGCs and collectors)
    with open('results.csv','w') as writer:
        writer.write(',')
        #loop to write FGCs names
        for i in range(len(FGC_Dic)):
            writer.write('%s,'%FGC_Dic[i])
        writer.write('\n')
        for i in range(len(predictions)):
            writer.write('%s,'%collectors[i])
            for j in range(len(predictions[i])):
                writer.write('%s,'%predictions[i][j])
            writer.write('\n')

    #get best 10 and save them besides desired value
    #writing on recommendations file
    writer=open('Recommendations.csv','w')
    #write header
    writer.write('Collector,Desired,Recomm_1,Recomm_2,Recomm_3,Recomm_4,Recomm_5,Recomm_6,Recomm_7,Recomm_8,Recomm_9,Recomm_10\n')
    #loop on all predictions
    for i in range(len(predictions)):
        #select best 20 from each prediction
        arr=np.array(predictions[i])
        BestTenIndecies=arr.argsort()[-20:][::-1]
        #write collector and desired product
        writer.write('%s,%s,' %(collectors[i] , FGC_Dic[np.argmax(Y[i])]))
        #loop and print first 10 recommendations
        for j in range(len(BestTenIndecies)):
            writer.write('%s,'%FGC_Dic[BestTenIndecies[j]])
        writer.write('\n')


    writer.flush()
    writer.close()

    #calculate accuracy of the system on recommendation 1,2,3 .... 20
    accuracy_List=[0]*20
    for i in range(len(predictions)):
        #select best 20 from each prediction
        arr=np.array(predictions[i])
        BestTwentyIndecies=arr.argsort()[-20:][::-1]
        desired= np.argmax(Y[i])
        #loop and print first 10 recommendations
        for j in range(len(BestTwentyIndecies)):
            if desired in BestTwentyIndecies[0:j+1]:
                accuracy_List[j]+=1.0

    #divide on size of predictions
    for j in range(len(BestTwentyIndecies)):
        accuracy_List[j]/=len(predictions)

    #write results on a file
    with open('Accuracy.csv','w') as writer:
        writer.write('#Recommendations,Accuracy\n')
        #loop and write accuracy of each group of recommendations
        for i in range(len(BestTwentyIndecies)):
            writer.write(str(i+1))
            writer.write(',')
            writer.write(str(accuracy_List[i]))
            writer.write('\n')




#function to load dictionary of save FGC names by the RNN_Model
def loadDictionary(path):
    dic=[]
    #read line by line and take name only (leave index)
    with open(path,'r') as reader:
        for line in reader:
            line=line.strip()
            if line=='':
                continue
            dic.append(line.split()[1])
    return dic


#the RNN_Model_Evaluation start running here
FGC_Dic=loadDictionary('FGC_Dictionary')
processor=DataProcessor() #create object
processor.Generate_X_Y_Vectors(path='Data.csv',FGC_Dic=FGC_Dic) #call generation function from the data processor file
X,Y,collectors=processor.X,processor.Y,processor.collectors #get X,Y,onehotMatrix matrices from the processor object
ApplyRNNModelEvaluation(X,Y,FGC_Dic,collectors) #call the RNN function
